<a href="https://colab.research.google.com/github/anshuldutt21/KeystrokeDynamics-Model/blob/main/CNN_Network_Keystroke_Dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network Model Experiments

This file experiments on multiple models of CNN to try and find out the best model. We have implemented VGG 16 model, Parallel block model and standard CNN model and compared them in our report. 

In [2]:
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.layers.experimental import preprocessing

dataset = pd.read_excel('/content/drive/MyDrive/DSL-StrongPasswordData.xls')
X = dataset.iloc[:,3:].values
y = dataset.iloc[:,0].values

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

# convert integers to dummy variables (i.e. one hot encoded)
Y = np_utils.to_categorical(encoded_Y)

X = X.astype('float64')

# Train and test pre processed separately

X_test = X[320:400,:]
Y_test = Y[320:400,:]
X_train = X[0:320,:]
Y_train = Y[0:320,:]


for x in range(1,51):
  X_test = np.concatenate((X_test, X[320 + 400*x:400 + 400*x,:]), axis = 0)
  Y_test = np.concatenate((Y_test, Y[320 + 400*x:400 + 400*x,:]), axis = 0)
  
  X_train = np.concatenate((X_train, X[400*x:320 + 400*x,:]), axis = 0)
  Y_train = np.concatenate((Y_train, Y[400*x:320 + 400*x,:]), axis = 0)

X_final_train = X_train
Y_final_train = Y_train

X_final_train, Y_final_train = shuffle(X_final_train, Y_final_train)
layer = preprocessing.Normalization()
layer.adapt(X_final_train)
X_final_train = layer(X_final_train)

layer.adapt(X_test)
X_test = layer(X_test)

print(X_final_train.shape)
print(X_test.shape)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
(16320, 31)
(4080, 31)


**Parallel Block Layer CNN Model**: The key idea is to apply
multiple rectangular kernels, instead of more typical square kernels. Specifically,
the width of all kernels is the same as the embedding size for each word, so the
output for each convolution is a one-dimension vector. Then multiple max-pooling
layers are used to process these vectors to yield one feature for each kernel. Finally,
these generated features are concatenated into a one-dimension vector, and multiple
fully-connected layers are used to produce the class prediction.

In [ ]:
from matplotlib import pyplot
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU
from keras.layers import Dropout                       #for random dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from keras import Input, Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Activation, Dropout, Flatten, Dense

#reshape data to fit model
X_final_train = np.reshape(X_final_train, (16320,1,31,1))
X_test = np.reshape(X_test, (4080,1,31,1))

kernel_size= {}
kernel_size[0]= [2,3,9]
kernel_size[1]= [3,3,8]
kernel_size[2]= [5,3,6]
kernel_size[3]= [7,3,4]
kernel_size[4]= [9,3,4]
kernel_size[5]= [11,3,4]

input_shape = (1,31,1)
initializer = keras.initializers.HeNormal()

inputs = Input(shape = (1,31,1))
convs = []

for k_no in range(len(kernel_size)):
    conv = Conv2D(32, kernel_size = (kernel_size[k_no][0], kernel_size[k_no][1]), padding='same', activation='relu')(inputs)
    pool = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv)
    for i in range(kernel_size[k_no][2]):
      pool = MaxPooling2D(pool_size=(2,2), padding = 'same')(pool)
    convs.append(pool)

if len(kernel_size) > 1:
    out = Concatenate()(convs)
else:
    out = convs[0]

conv_model = Model(inputs, out)

model = Sequential()
model.add(conv_model)
model.add(Flatten(input_shape=input_shape))

model.add(Dense(128, kernel_initializer=initializer, bias_initializer='zeros'))
model.add(Activation('relu'))

model.add(Dense(256, kernel_initializer=initializer, bias_initializer='zeros'))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(51, activation='softmax'))

# Compile model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
  
# fit model
history = model.fit(X_final_train, Y_final_train, validation_data=(X_test, Y_test), epochs=200, batch_size = 32)

# evaluate the model
_, train_acc = model.evaluate(X_final_train, Y_final_train, verbose=0)
_, test_acc = model.evaluate(X_test, Y_test, verbose=0)

print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
print("\n\n")
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()


**VGG - 16 Model:** It consists of 13 convolutional l layers with 3x3 filter size, 5 subsampling/ max pooling layer with size of 2x2, two fully connected layers with activation function and softmax function. Each block is made by consecutive 3 × 3 convolutions and followed by a max pooling layer. The number of filter is gradually increased from size of 64,128,256,512 and 512. To avoid the problem of over-fitting, we use dropout.

In [ ]:
from matplotlib import pyplot
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU
from keras.layers import Dropout                       #for random dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from keras import Input, Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Activation, Dropout, Flatten, Dense

#reshape data to fit model
X_final_train = np.reshape(X_final_train, (16320,1,31,1))
X_test = np.reshape(X_test, (4080,1,31,1))

input_shape = (1,31,1)

inputs = Input(shape = (1,31,1))
initializer = keras.initializers.HeNormal()

conv1 = Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(inputs)
conv1 = Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv1)
conv1 = BatchNormalization()(conv1)
max1 = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv1)

conv2 = Conv2D(128, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(max1)
conv2 = Conv2D(128, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv2)
conv2 = BatchNormalization()(conv2)
max2 = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv2)

conv3 = Conv2D(256, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(max2)
conv3 = Conv2D(256, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv3)
conv3 = Conv2D(256, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv3)
conv3 = BatchNormalization()(conv3)
max3 = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv3)

conv4 = Conv2D(512, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(max3)
conv4 = Conv2D(512, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv4)
conv4 = Conv2D(512, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv4)
conv4 = BatchNormalization()(conv4)
max4 = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv4)

conv5 = Conv2D(512, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(max4)
conv5 = Conv2D(512, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv4)
conv5 = Conv2D(512, (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv4)
conv5 = BatchNormalization()(conv5)
max5 = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv5)

flat = Flatten()(max5)

den1 = Dense(512, activation = 'relu')(flat)
den1 = Dropout(0.5)(den1)
out1 = Dense(51, activation ='softmax')(den1)

model = Model(inputs, out1)

# Compile model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy', keras.metrics.Precision(),keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.Recall()])
  
# fit model
history = model.fit(X_final_train, Y_final_train, validation_data=(X_test, Y_test), epochs=200, batch_size = 32)

# evaluate the model
_, test_acc = model.evaluate(X_test, Y_test, verbose=0)

print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
print("\n\n")
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()


**Standard CNN Network:** The CNN model consists of four hidden layers (three convolutional sub sampling layer pair and one fully connected layer), one input layer and one output layer.  The convolutional- sub sampling layer use the size 3x3 stride lengths followed by 2x2 regions. The number of filter is gradually increased from size of 64,128 and 256. The fully connected layer contains 1024 neurons with Rectified Linear Unit(ReLU) as activation funciton.

In [ ]:
from matplotlib import pyplot
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU
from keras.layers import Dropout                       #for random dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from keras import Input, Sequential
from keras.models import Model
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Activation, Dropout, Flatten, Dense


input_shape = (1,31,1)

inputs = Input(shape = (1,31,1))
initializer = keras.initializers.HeNormal()

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Define the K-fold Cross Validator
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1
Xshape = X.shape
Yshape = Y.shape

for train, test in kfold.split(X, Y):
  Xtrain = np.empty(Xshape)
  Xtrain = Xtrain[0:0,:]
  
  Ytrain = np.empty(Yshape)
  Ytrain = Ytrain[0:0,:]
  
  Xtest = np.empty(Xshape)
  Xtest = Xtest[0:0,:]
  
  Ytest = np.empty(Yshape)
  Ytest = Ytest[0:0,:]

  for i in train:
    Xtrain = np.vstack([Xtrain, X[i,:]])
    Ytrain = np.vstack([Ytrain, Y[i,:]])

  for i in test:
    Xtest = np.vstack([Xtest, X[i,:]])
    Ytest = np.vstack([Ytest, Y[i,:]])

  print(Xtrain)
  layer = preprocessing.Normalization()
  layer.adapt(Xtrain)
  Xtrain = layer(Xtrain)
  layer.adapt(Xtest)
  Xtest = layer(Xtest)
  
  Xtrain = np.reshape(Xtrain, (16320,1,31,1))
  Xtest = np.reshape(Xtest, (4080,1,31,1))


  conv1 = Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(inputs)
  conv1 = BatchNormalization()(conv1)
  conv1 = Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv1)
  conv1 = BatchNormalization()(conv1)
  conv1 = Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu', kernel_initializer=initializer)(conv1)
  conv1 = BatchNormalization()(conv1)
  max1 = MaxPooling2D(pool_size=(2,2), padding = 'same')(conv1)

  flat = Flatten()(max1)

  den1 = Dense(512, activation = 'relu')(flat)
  den1 = BatchNormalization()(den1)
  den1 = Dropout(0.5)(den1)
  out1 = Dense(51, activation ='softmax')(den1)

  model = Model(inputs, out1)

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Compile model
  optimizer = keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy', keras.metrics.Precision(),keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.Recall()])
    
  # fit model
  history = model.fit(Xtrain, Ytrain, validation_data=(Xtest, Ytest), epochs=200, batch_size = 32)

  # evaluate the model
  scores = model.evaluate(Xtest, Ytest, verbose=0)
  
  print(scores)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # plot loss during training
  pyplot.subplot(211)
  pyplot.title('Loss')
  pyplot.plot(history.history['loss'], label='train')
  pyplot.plot(history.history['val_loss'], label='test')
  pyplot.legend()
  print("\n\n")
  # plot accuracy during training
  pyplot.subplot(212)
  pyplot.title('Accuracy')
  pyplot.plot(history.history['accuracy'], label='train')
  pyplot.plot(history.history['val_accuracy'], label='test')
  pyplot.legend()
  pyplot.show()

  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')